In [15]:
import classes_taskonomy

# modified visualpriors library
from transforms import VisualPriorRepresentation, VisualPriorPredictedLabel
from taskonomy_network import TaskonomyEncoder, TaskonomyDecoder

import os
import collections
import numpy as np # type: ignore
import pandas as pd # type: ignore

import matplotlib.pyplot as plt # type: ignore
import torchsummary # type: ignore

from PIL import Image # type: ignore
from scipy.stats import pearsonr # type: ignore
from scipy.io import savemat, loadmat # type: ignore

import torch.utils.model_zoo # required to load nets # type: ignore
import torchvision.transforms.functional as TF # type: ignore
from torch import nn # type: ignore
import torchvision.models # type: ignore
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor # type: ignore

#%load_ext autoreload
#%autoreload 2

In [11]:
from transforms import VisualPrior

In [10]:
_, eval_nodes = get_graph_node_names(net)

for node in eval_nodes:
    print(node, end='   ')

return_nodes = { node:node for node in eval_nodes if "conv" in node or 'fc' in node}

net_tweaked = create_feature_extractor(net, return_nodes=return_nodes)

x   pad   conv1   bn1   relu   maxpool   layer1.0.conv1   layer1.0.bn1   layer1.0.relu   layer1.0.conv2   layer1.0.bn2   layer1.0.relu_1   layer1.0.conv3   layer1.0.bn3   layer1.0.downsample.0   layer1.0.downsample.1   layer1.0.add   layer1.0.relu_2   layer1.1.conv1   layer1.1.bn1   layer1.1.relu   layer1.1.conv2   layer1.1.bn2   layer1.1.relu_1   layer1.1.conv3   layer1.1.bn3   layer1.1.add   layer1.1.relu_2   layer1.2.conv1   layer1.2.bn1   layer1.2.relu   layer1.2.conv2   layer1.2.bn2   layer1.2.relu_1   layer1.2.conv3   layer1.2.bn3   layer1.2.downsample.0   layer1.2.add   layer1.2.relu_2   layer2.0.conv1   layer2.0.bn1   layer2.0.relu   layer2.0.conv2   layer2.0.bn2   layer2.0.relu_1   layer2.0.conv3   layer2.0.bn3   layer2.0.downsample.0   layer2.0.downsample.1   layer2.0.add   layer2.0.relu_2   layer2.1.conv1   layer2.1.bn1   layer2.1.relu   layer2.1.conv2   layer2.1.bn2   layer2.1.relu_1   layer2.1.conv3   layer2.1.bn3   layer2.1.add   layer2.1.relu_2   layer2.2.conv1   layer2.

# Load images

In [47]:
loadmat("data/stimuli_places1_resized/Places365_val_00001153.mat")['im']
import subprocess
subprocess.call("curl -O https://raw.githubusercontent.com/StanfordVL/taskonomy/master/taskbank/assets/test.png", shell=True)
img = Image.open('./data/stimuli_places1/Places365_val_00001153.jpg')
TF.to_tensor(img)

img = TF.to_tensor(TF.resize(img, 256)) * 2 - 1 # resize
img = img.unsqueeze_(0)

# load model
task = 'autoencoding'

VisualPriorRepresentation._load_unloaded_nets([task]) # preload nets
net = VisualPriorRepresentation.feature_task_to_net[task] # loads encoder only for representations
torchsummary.summary(net, (3, 256, 256))

array([[[ 24.440285 ,  22.75106  ,  11.813852 ],
        [ 28.500277 ,  27.750185 ,  10.516175 ],
        [ 46.8996   ,  46.354485 ,  20.093699 ],
        ...,
        [153.51295  , 112.667366 ,  73.45321  ],
        [114.17797  ,  74.978455 ,  48.79689  ],
        [128.2486   ,  97.46462  ,  79.03427  ]],

       [[ 58.05664  ,  61.188354 ,  38.276947 ],
        [ 54.278408 ,  57.572754 ,  31.45102  ],
        [ 52.65642  ,  55.115265 ,  25.720566 ],
        ...,
        [120.24372  ,  86.13118  ,  58.90142  ],
        [ 59.342533 ,  29.822533 ,  15.940515 ],
        [ 42.107304 ,  22.485958 ,  14.650325 ]],

       [[ 36.602886 ,  44.835506 ,  11.999634 ],
        [ 41.501434 ,  49.640537 ,  15.932427 ],
        [ 40.36918  ,  46.6992   ,  15.063794 ],
        ...,
        [149.50587  , 125.88573  , 112.681854 ],
        [ 44.91447  ,  27.869555 ,  25.124537 ],
        [  7.946582 ,   2.764554 ,   4.4361887]],

       ...,

       [[  5.589234 ,  10.654429 ,   4.7645097],
        [  

In [ ]:
# rescale to [-1, 1]

# Inspect model output

In [ ]:
"""Does the manually extracted network work as expected ?"""

In [13]:
net

TaskonomyEncoder(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bia

# Compute integration 

In [6]:
places_images = classes_taskonomy.ImageDatasetTaskonomy('./data/stimuli_places1', beauty_ratings_path='./behavior/ratings_study1.csv')
integration_calc = classes_taskonomy.IntegrationCalculatorTaskonomy(net_tweaked, return_nodes)
result = classes_taskonomy.calculate_dataset_integration(places_images, integration_calc)

In [9]:
result

,conv1,layer1.0.conv1,layer1.0.conv2,layer1.0.conv3,layer1.1.conv1,layer1.1.conv2,layer1.1.conv3,layer1.2.conv1,layer1.2.conv2,layer1.2.conv3,...,layer3.5.conv3,layer4.0.conv1,layer4.0.conv2,layer4.0.conv3,layer4.1.conv1,layer4.1.conv2,layer4.1.conv3,layer4.2.conv1,layer4.2.conv2,layer4.2.conv3
0,-0.901755,-0.927244,-0.906128,-0.892599,-0.924830,-0.857169,-0.839657,-0.941186,-0.896023,-0.852555,...,-0.859965,-0.995464,-0.858975,-0.725659,-0.895166,-0.546067,-0.366636,-0.620608,-0.348664,-0.371284
1,-0.977258,-0.979798,-0.977366,-0.970708,-0.973443,-0.941702,-0.918240,-0.974717,-0.968564,-0.947710,...,-0.938060,-0.996411,-0.912282,-0.805249,-0.957089,-0.706170,-0.470574,-0.755524,-0.356730,-0.136613
2,-0.985005,-0.988065,-0.982306,-0.978973,-0.980807,-0.959042,-0.937909,-0.980815,-0.977924,-0.965159,...,-0.932084,-0.996749,-0.916451,-0.840916,-0.964348,-0.723489,-0.497462,-0.776550,-0.360851,-0.148527
3,-0.969972,-0.971745,-0.968256,-0.959294,-0.971802,-0.943989,-0.924279,-0.974445,-0.967046,-0.948057,...,-0.903482,-0.997028,-0.915133,-0.862330,-0.956345,-0.715045,-0.582552,-0.792562,-0.507871,-0.420497
4,-0.982985,-0.985039,-0.980419,-0.975774,-0.981731,-0.966215,-0.950199,-0.980384,-0.976013,-0.963792,...,-0.902900,-0.996946,-0.916627,-0.884209,-0.963456,-0.755956,-0.626542,-0.805912,-0.571683,-0.499548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,-0.967946,-0.978558,-0.971832,-0.965229,-0.966798,-0.933618,-0.914758,-0.967229,-0.955363,-0.933746,...,-0.926590,-0.995430,-0.902838,-0.806128,-0.947222,-0.727485,-0.504238,-0.768200,-0.392083,-0.184156
246,-0.971098,-0.974567,-0.971881,-0.963860,-0.971524,-0.946957,-0.928143,-0.972739,-0.964131,-0.943081,...,-0.925275,-0.996563,-0.914013,-0.855937,-0.954991,-0.736623,-0.608247,-0.798507,-0.543984,-0.475614
247,-0.970213,-0.975484,-0.968955,-0.964136,-0.970212,-0.949807,-0.937206,-0.966894,-0.957912,-0.952441,...,-0.924776,-0.993317,-0.912605,-0.901262,-0.942384,-0.802596,-0.611354,-0.816946,-0.436379,-0.208226
248,-0.991683,-0.993783,-0.991791,-0.989363,-0.987538,-0.974684,-0.956181,-0.986093,-0.987357,-0.976262,...,-0.924651,-0.997633,-0.923652,-0.856157,-0.972104,-0.729208,-0.493144,-0.781738,-0.362910,-0.120459
